In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME ##### ->FIXED JB 4/2/2023
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import Crud



###########################
# Data Manipulation / Model
###########################
# Current credentials for using CRUD.py
username = "aacuser"
password = "Gil"
shelter = Crud(username, password, 41428, "AAC", "animals")


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

# Creates an index id for selectable rows
df["id"] = df.index
df.set_index("id", inplace = True, drop = False)

# The following ensures CRUD and MongoDB are functional: 
# print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Adds Grazioso Salvare Logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    # Displays logo and scales it down to fit screen
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={
                    'height' : '15%',
                    'width' : '15%',}
                        )),
    html.Center(html.B(html.H3('GRAZIOSO SALVARE-DASH TABLE'))),
    html.Center(html.B(html.H5('By: John Brungard'))),
    html.Hr(),
    html.Div(
        #Radio Items to select the rescue filter options
        dcc.Dropdown(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'DRIT'},
            {'label': 'Reset', 'value': 'RESET'}
            ],
            placeholder = 'Please select an option...',          
        )
    ),
    html.Hr(),

      
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns if i != 'id'
        ],
        data=df.to_dict('records'),
        # Features for data table
        
        # Can alter data in table if True
        editable = False,
        
        # Filter table by column
        filter_action = "native",
        
        # Sort table by column
        sort_action = "native",
        
        sort_mode = "multi",
        
        # Select columns. Can be "single", False or "multi"
        # Single means only 1 can be selected at a time.
        column_selectable= False,
        
        # Select rows. Can be "single", False or "multi"
        # Multi means more than 1 can be selected at a time.
        row_selectable= "single",
        
        # Rows cannot be deleted
        row_deletable = False,
        
        selected_columns = [],
        selected_rows = [],
        
        # Paging front-end
        page_action = "native",
        
        # Adds vertical & horizontal scroller if needed
        # Note: If you want height to be fixed add in "height" : "300px" 
        # Each row is 30px so 10 rows + 1 header would be 330px
        # If omitted, height is dynamic up to 250 rows
        style_table={'height': '226px', 'overflowY': 'auto', 'overflowX': 'auto'},
        
        # Provides padding for all columns
        # Used to accommodate the sort widget        
        style_cell = {'padding-left':'8px', 'padding-right': '8px', 'text-align': 'center'},
        
        # Styles header background and text
        style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
        },
        # Styles data background and text
        style_data={
        'backgroundColor': 'rgb(235, 235, 235)',
        'color': 'black'
        },
        
        # Adjusts background of filter cells and table border color
        style_cell_conditional = [{'if': {'state':'active'},
                                  'backgroundColor': 'rgba(255, 255, 255, 0.3)',
                                   'border': '1px solid rgb(125, 5, 5)'}],
    
        # Set page to be first viewed when launched. Ex: 0 = page 1, 1 = page 2, etc.
        page_current = 0,
        
        # Total rows allowed per page
        page_size = 50,
           
        # Makes headers fixed
        # Making static headers can pair well with multi-line data & virtualization
        # Can add after True: 'data' : row# to make a column fixed when scrolling
        #fixed_rows={'headers': True},
        
        # Sets a fixed width of the columns
        #style_cell={'minWidth': "200px", 'maxWidth': "200px", 'width': "200px"},
        
        # Wraps large text onto multiple lines
        # Would use this as an alternative to horizontal scrolling or if text is really big
        #style_data={'whiteSpace': 'normal', 'height': 'auto','lineHeight': '15px'},
        
        # Renders rows as you scroll instead of all at once
        # Must have fixed headers otherwise dynamic resizing occurs
        #virtualization=True,
        
        

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will filter interactive data table with MongoDB queries
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
#Updates the dashboard results with the given filter options
def update_dashboard(filter_type):
    # If user selects "Water Rescue"
    if filter_type == 'WR':
        # Gathers the preferred variables of a Water Rescue Animal
        # Note: There are multiple variations of Newfoundland. All were added into this section
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                        {'breed': {'$in': [
                                                            'Labrador Retriever Mix',
                                                            'Chesa Bay Retr Mix',
                                                            'Newfoundland Mix',
                                                            'Newfoundland/Labrador Retriever',
                                                            'Labrador Retriever/Newfoundland',
                                                            'Newfoundland/Australian Cattle Dog',
                                                            'Newfoundland/Great Pyrenees']
                                                        }},
                                                        {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                        }]
                                            }))
    # If user selects "Mountain/Wilderness Rescue" 
    elif filter_type == 'MWR':
        # Gathers the preferred variables of a Mountain/Wilderness Rescue Animal
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                        {'breed': {'$in': [
                                                            'German Shepherd',
                                                            'Alaskan Malamute',
                                                            'Old English Sheepdog',
                                                            'Siberian Husky',
                                                            'Rottweiler']
                                                        }},
                                                        {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                        }]
                                            }))
    # If user selects "Disaster Rescue/Individual Tracking" 
    elif filter_type == 'DRIT':
        # Gathers the preferred variables of a Disaster Rescue/Individual Tracking Animal
        df = pd.DataFrame.from_records(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                        {'breed': {'$in': [
                                                            'Doberman Pinscher',
                                                            'German Shepherd',
                                                            'Golden Retriever',
                                                            'Bloodhound',
                                                            'Rottweiler']
                                                        }},
                                                        {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                        }]
                                            }))
    #If user selects 'Reset', the table displays all contents of the DB again
    elif filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))

    # Reconfigures and returns the contents found by the selected filter into the dashtable
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns if i != 'id']
    data=df.to_dict('records')
        
    return (data,columns)
                                        

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if selected_rows is None:
        return dash.no_update
    return [
        {"if": {"filter_query": "{{id}} ={}".format(i)}, "backgroundColor": "yellow",}
        for i in selected_rows
    ]


# This callback will display a pie chart based on the filter on the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # Obtains the current filter
    dff = pd.DataFrame.from_dict(viewData)
    
    # Creates the key and %'s for each breed in the viewData
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    # Creates the pie chart
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff, 
                values = values,
                names = names, 
                color_discrete_sequence = px.colors.sequential.Plasma,
                width = 750, 
                height = 375, 
                title = "Filtered Animal By Breed and Quantity"
            )
        )
    ]

# This callback will display a map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")]
)    
def update_map(viewData, selected_rows):
    # Reverses dict into Pandas DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Makes map appear with first animal as a marker
    # This is a default and will be overridden when row is selected
    if selected_rows == []:
        selected_rows = [0]
        
    return [
        # Dictates the size, view, and magnification of the map
        dl.Map(style={'width': '750px', 'height': '375px'}, center=[30.75,-97.48], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            
            # Selects positioning where marker will be placed
            dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                # Shows the selected row/column data when hovering over marker
                dl.Tooltip(dff.iloc[selected_rows,2]),
                # Presents information about data when clicking marker
                dl.Popup([
                        html.H4("Name"),
                        html.P(dff.iloc[selected_rows,9]),
                        html.H4("Sex"),
                        html.P(dff.iloc[selected_rows,12]),
                        html.H4("Breed"),
                        html.P(dff.iloc[selected_rows,4]),
                        html.H4("Outcome"),
                        html.P(dff.iloc[selected_rows,11]),
                        html.H4("Age_Outcome_Weeks"),
                        html.P(dff.iloc[selected_rows,15])
                ])
            ])
        ])
    ]

app
